In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
DATA_DIR = "/content/drive/MyDrive/plant_disease_project"
!mkdir -p $DATA_DIR


In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset -p $DATA_DIR

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
100% 2.69G/2.70G [00:32<00:00, 113MB/s]
100% 2.70G/2.70G [00:32<00:00, 88.4MB/s]


In [ ]:
!ls $DATA_DIR

'new plant diseases dataset(augmented)'   new-plant-diseases-dataset.zip
'New Plant Diseases Dataset(Augmented)'   test


In [ ]:

import tensorflow as tf
import os

# Paths to raw data (train/valid folders)
train_dir = "//content/drive/MyDrive/plant_disease_project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/content/drive/MyDrive/plant_disease_project/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

# Preprocess training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",   # one-hot labels
    batch_size=32,
    image_size=(128, 128),      # resize all images
    shuffle=True
)

# Preprocess validation data
valid_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False
)

# Optional: normalize pixel values (0–1)
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))

print("✅ Preprocessing complete! Data ready for training.")


Found 70295 files belonging to 38 classes.
Found 15643 files belonging to 35 classes.
✅ Preprocessing complete! Data ready for training.


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

# Where to save processed dataset
output_dir = "/content/drive/MyDrive/plant_disease_project/processed_dataset"
os.makedirs(output_dir, exist_ok=True)

def preprocess_and_save(input_dir, output_subdir):
    output_path = os.path.join(output_dir, output_subdir)
    os.makedirs(output_path, exist_ok=True)

    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        save_class_path = os.path.join(output_path, class_name)
        os.makedirs(save_class_path, exist_ok=True)

        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            img_path = os.path.join(class_path, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))
                cv2.imwrite(os.path.join(save_class_path, img_name), img)
            except:
                continue

# Process train and valid sets
preprocess_and_save(train_dir, "train")
preprocess_and_save(valid_dir, "valid")

print("✅ Resized dataset saved to:", output_dir)


Processing Apple___Apple_scab:  78%|███████▊  | 1580/2016 [08:14<02:02,  3.55it/s]